# Era3D notebook for Google Colab

Requres a hardware accelerator: T4 GPU.

How-to video:  
<a href="https://youtu.be/1jcSA5TjZ1k"><img src="https://i3.ytimg.com/vi/1jcSA5TjZ1k/maxresdefault.jpg" width=200) /></a>


In [ ]:
#@markdown # Installation
#@markdown
%%time
home_dir = '/content'
python = 'python'
pip = 'pip'
install_era_to_google_drive = False #@param {type:"boolean"}
#@markdown * Uses about 13 GB and has not been tested yet

install_libraries_from_drive = True #@param {type:"boolean"}
#@markdown * Uses about 5 GB but significantly speeds up installation

from google.colab import drive
import os
import stat
if (install_era_to_google_drive or install_libraries_from_drive):
  drive.mount('/content/drive')
  os.makedirs('/content/drive/MyDrive/Era3D_Install', exist_ok=True)

if (install_era_to_google_drive):
  home_dir = '/content/drive/MyDrive/Era3D_Install'

def find_bin_folders(folder_path):
    bin_folders = []
    for root, dirs, files in os.walk(folder_path):
        for dir_name in dirs:
            if dir_name == 'bin':
                bin_folders.append(os.path.join(root, dir_name))
    return bin_folders

def installEra(home_dir, install_era_to_google_drive):
  %cd {home_dir}
  !git lfs clone https://huggingface.co/spaces/pengHTYX/Era3D_MV_demo/
  %cd {home_dir}/Era3D_MV_demo
  !git clone https://github.com/wandaweb/Era3D
  !mv {home_dir}/Era3D_MV_demo/Era3D/instant-nsr-pl {home_dir}/Era3D_MV_demo/instant-nsr-pl
  !mv {home_dir}/Era3D_MV_demo/Era3D/blender {home_dir}/Era3D_MV_demo/
  !mv {home_dir}/Era3D_MV_demo/Era3D/data_lists {home_dir}/Era3D_MV_demo/
  !mv {home_dir}/Era3D_MV_demo/Era3D/node_config {home_dir}/Era3D_MV_demo/
  %cd {home_dir}/Era3D_MV_demo/
  if not os.path.exists(f'{home_dir}/Era3D_MV_demo/MacLab-Era3D-512-6view/model_index.json'):
    get_ipython().system(f'rm -rf {home_dir}/Era3D_MV_demo/MacLab-Era3D-512-6view')
  !git lfs clone https://huggingface.co/pengHTYX/MacLab-Era3D-512-6view

def installLibraries(home_dir, python, pip):
  %cd {home_dir}
  !{python} -m pip install --force-reinstall torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 xformers --index-url https://download.pytorch.org/whl/cu118
  !{pip} install pygments
  if not os.path.exists(f'tinycudann-1.7-cp310-cp310-linux_x86_64.whl'):
    get_ipython().system('wget "https://j2q5.c17.e2-1.dev/download/pogscafe/tinycudann-1.7-cp310-cp310-linux_x86_64.whl"')
  !{pip} install tinycudann-1.7-cp310-cp310-linux_x86_64.whl
  !{pip} install git+https://github.com/NVlabs/nvdiffrast
  %cd {home_dir}/Era3D_MV_demo/
  !{pip} install -r requirements.txt
  !{pip} install fire spaces

  !{pip} install moviepy trimesh rembg icecream
  !rm -rf /opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info
  %cd {home_dir}/Era3D_MV_demo/Era3D/instant-nsr-pl
  !{pip} install -r requirements.txt
  !{pip} install --force-reinstall numpy
  !{pip} install --force-reinstall pydantic
  !{pip} install tensorflow[and-cuda]

installEra(home_dir, install_era_to_google_drive)
if (install_libraries_from_drive):
    print('installing venv')
    get_ipython().system('touch /content/use_venv')
    venv_parent = '/content'
    print(venv_parent)
    os.chdir(venv_parent)
    get_ipython().system('pip install virtualenv')
    python = f'{venv_parent}/venv/bin/python'
    pip = f'{venv_parent}/venv/bin/pip'
    if not os.path.exists('/content/venv'):
      if not os.path.exists('/content/drive/MyDrive/Era3D_Install/venv-colab.tar.gz'):
        get_ipython().system('wget "https://j2q5.c17.e2-1.dev/download/pogscafe/era3d_kaggle/venv-colab.tar.gz" -O "/content/drive/MyDrive/Era3D_Install/venv-colab.tar.gz"')
        # another option if the above link is broken
        # get_ipython().system('wget "https://q4j3.c11.e2-5.dev/downloads/pogscafe/venv-colab.tar.gz" -O "/content/drive/MyDrive/Era3D_Install/venv-colab.tar.gz"')
      get_ipython().system('tar -xvf /content/drive/MyDrive/Era3D_Install/venv-colab.tar.gz -C /content/')
    get_ipython().system('unlink /content/venv/bin/python')
    get_ipython().system('unlink /content/venv/bin/python3')
    get_ipython().system('ln -s /content/venv/bin/python3.10 /content/venv/bin/python')
    get_ipython().system('ln -s /content/venv/bin/python3.10 /content/venv/bin/python3')
    bin_folders = find_bin_folders(f'{venv_parent}/venv')
    if bin_folders:
      print("Found 'bin' folders:")
      for bin_folder in bin_folders:
        print(bin_folder)
        for filename in os.listdir(bin_folder):
            file_path = os.path.join(bin_folder, filename)
            if os.path.isfile(file_path):
                current_permissions = os.stat(file_path).st_mode
                # Add execute permissions for the user, group, and others
                os.chmod(file_path, current_permissions | stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH)
else:
  get_ipython().system('rm /content/use_venv')
  installLibraries(home_dir, python, pip)

if (install_era_to_google_drive):
    get_ipython().system(f'ln -s {home_dir}/Era3D_MV_demo /content/')

!apt install -y mesa-utils freeglut3-dev libnvidia-gl-470 libegl1-mesa-dev
!apt-get install ninja-build

import fileinput
import os
for line in fileinput.input('/content/Era3D_MV_demo/app.py', inplace=True):
    if ('demo.queue().launch(share=True, max_threads=80)' in line):
       line = line.replace('demo.queue().launch(share=True, max_threads=80)', 'demo.queue().launch(share=False, max_threads=20)')
    print(line, end='')



---



# Start the WebUI

You can use the web UI to generate images and normals for mesh creation. When finished, move to the next step to generate the 3d model.


In [ ]:
#@markdown # Option 1: Start the Web UI with Localtunnel

%cd /content/Era3D_MV_demo
!npm install -g localtunnel
python = 'python'
import os
if os.path.exists('/content/use_venv'):
  python = '/content/venv/bin/python3.10'

# @markdown
# @markdown Copy the IP address shown in the output above the line
# @markdown "your url is: https://some-random-words.loca.lt"
!wget -q -O - ipv4.icanhazip.com

#@markdown Wait for the line that says "Running on http://127.0.0.1:7860"

#@markdown Click the localtunnel url and paste the IP you copied earlier to the "Endpoint IP" text field
!lt --port 7860 --local_https False & {python} /content/Era3D_MV_demo/app.py

#@markdown If the tunnel disconnects during generation, check if the outputs were successfuly generated in the folder /content/Era3D_MV_demo/mv_res


In [ ]:
#@markdown # Option 2: Start the Web UI with ngrok
!pip install pyngrok

from pyngrok import ngrok, conf
import fileinput
import sys

ngrok_token = "" #@param {type:"string"}
ngrok_domain = "" #@param {type:"string"}
#@markdown - Add ngrok token (obtainable from https://ngrok.com)

python = 'python'
import os
if os.path.exists('/content/use_venv'):
  python = '/content/venv/bin/python'

if ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(7860 , pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token),
                    bind_tls=True, domain=ngrok_domain).public_url
  print(srv)
  get_ipython().system(f"{python} /content/Era3D_MV_demo/app.py")
else:
  print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the ngrok_token field.')



---



In [ ]:
#@markdown # Generate the mesh
#@markdown Check the box below to store output models in Google Drive.
use_google_drive = True #@param {type:"boolean"}

#@markdown Leave the box below empty to use the latest generated images, or enter the full path to the
#@markdown images you want turned into a mesh.

input_path = '' #@param {type:"string"}
#@markdown Example: input_path = "/content/Era3D_MV_demo/mv_res/cropsize-420-cfg3.0/scene@20240605-211017"

# --------------------------------------------------

import os

def find_highest_second_level_subfolder(path):
    highest_folder = None
    parent_folder = None
    for root, dirs, files in os.walk(path):
        depth = root[len(path):].count(os.sep)
        if depth == 1:
            for d in dirs:
                if highest_folder is None or d > highest_folder:
                    highest_folder = d
                    parent_folder = root
    return highest_folder, parent_folder

input_dir = ''
if input_path == '':
  mv_res_path = '/content/Era3D_MV_demo/mv_res'
  highest_folder, parent_folder = find_highest_second_level_subfolder(mv_res_path)
  input_dir=os.path.join(parent_folder, highest_folder)
  print(input_dir)
input_dir = input_dir.split('mv_res/')[1]
outdir = '/content/output'
import os
if use_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')
  os.makedirs('/content/drive/MyDrive/Era3D-Output', exist_ok=True)
  outdir = '/content/drive/MyDrive/Era3D-Output'

os.makedirs(outdir, exist_ok=True)
%cd /content/Era3D_MV_demo/instant-nsr-pl
!{python} launch.py  --config configs/neuralangelo-ortho-wmask.yaml --gpu 0 \
    --train dataset.root_dir=../mv_res dataset.scene={input_dir} \
    --exp_dir={outdir}
print('Finished')

/content/Era3D_MV_demo/mv_res/cropsize-420-cfg3.0/scene@20240612-025836
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/Era3D_MV_demo/instant-nsr-pl
2024-06-12 03:06:42.673338: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 03:06:44.399799: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Global seed set to 42
Using finite difference to compute gradients with eps=progressive
Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` w